<a href="https://colab.research.google.com/github/anjaleeDS/Sandbox/blob/master/Rose_WEBscrapingNotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas way to scrape and clean UC Davis rose catalog

In [2]:
import lxml.etree as xml
import requests  # go out and fetch itnative python
import pandas as pd
import numpy as np
import re
import time
import requests


In [3]:
from google.colab import drive
drive.mount('drive')
%cd drive/My Drive/Colab Notebooks/webscraping


Mounted at drive
/content/drive/My Drive/Colab Notebooks/webscraping


In [4]:
url = "https://fps.ucdavis.edu/roses/collection.cfm"
requests.get(url)

<Response [200]>

Get the table data

In [140]:
r=requests.get(url)
df_list= pd.read_html(r.text)


make into a dataframe

In [202]:
df = pd.DataFrame(data=df_list[1],copy=True)
df.shape

(821, 6)

clean the headers

In [203]:
df.columns = df.columns.str.strip() # blanks
df.columns = df.columns.str.replace(r"[^a-zA-Z\d\_]+", "")

df.columns


Index(['Unnamed0', 'Cultivar', 'FlowerColor', 'FlowerColor1', 'PlantType',
       'Fragrance'],
      dtype='object')

take out columns that have no data


In [204]:
for col in df.columns:
  if df[col].count()==0:
    # print(col)
    df = df.drop(col, axis=1)  

df.columns


Index(['Cultivar', 'FlowerColor1', 'PlantType', 'Fragrance'], dtype='object')

In [205]:
df.shape[1]

4

get the detail page URLs

In [206]:
d_url = (re.findall('cultivar.cfm\?varietyid=\d+', r.text)) # get all the onclicks
d_url = pd.DataFrame(data=d_url) # stick them in a df

df = (pd.concat([df,d_url],axis=1)) # concat the dfs together
df = df.rename({df.columns[df.shape[1]-1] :'details'},axis=1) # rename that concated column to the correct column name

df

,Cultivar,FlowerColor1,PlantType,Fragrance,details
0,A Shropshire Lad ®,light pink,Shrub,moderate,cultivar.cfm?varietyid=3568
1,Abbaye De Cluny ™,apricot & apricot blend,Hybrid Tea & Climbing Hybrid Tea,slight,cultivar.cfm?varietyid=2132
2,About Face ™,orange & orange blend,Grandiflora & Climbing Grandiflora,none,cultivar.cfm?varietyid=2774
3,Above All ™,orange & orange blend,Large-Flowered Climber,moderate - fruity,cultivar.cfm?varietyid=4973
4,Above and Beyond ™,apricot & apricot blend,Large-Flowered Climber,moderate - spice,cultivar.cfm?varietyid=6254
...,...,...,...,...,...
816,Young Lycidas,deep pink,Shrub,intense - delicious tea and old rose with hint...,cultivar.cfm?varietyid=6060
817,You're the One ™,pink blend,Miniature & Climbing Miniature,slight,cultivar.cfm?varietyid=5005
818,Yuletide,medium red,Hybrid Tea & Climbing Hybrid Tea,slight,cultivar.cfm?varietyid=1631
819,Yves Piaget ®,medium pink,Hybrid Tea & Climbing Hybrid Tea,intense,cultivar.cfm?varietyid=1704


In [207]:
# make all of the details into urls with detail
df['details'] = df.apply(lambda x:"https://fps.ucdavis.edu/roses/"+  x['details'], axis=1)

fetch the details from the detail page

In [210]:
def add_details(detail_url, df):
  d = requests.get(detail_url)
  return(pd.read_html(d.text))

In [211]:
df['details'][0]

'https://fps.ucdavis.edu/roses/cultivar.cfm?varietyid=3568'

In [213]:
stuff = add_details(df['details'][0], df)

In [222]:
stuff.shape

AttributeError: ignored